# **Random Forest**

**Reference: Manual/Tuned**

Base: 0.69893 / 0.71938

New: **0.68189** / 0.69952


## **Setup**

In [1]:
!git clone https://github.com/yeray142/AIML_project

Cloning into 'AIML_project'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 53 (delta 17), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [1]:
cd AIML_project

[WinError 2] El sistema no puede encontrar el archivo especificado: 'AIML_project'
c:\Users\david\Documents\GitHub\AIML_project\notebooks


In [38]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/new_train.csv", index_col=0)
test = pd.read_csv("../data/new_test.csv", index_col=0)

In [39]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

## **Data preprocessing**

In [40]:
train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
0,F,N,N,202500.0,Commercial associate,Higher education,Married,Municipal apartment,0,0,...,38,7.0,1.0,12,0.0,0.0,25,6.0,0.0,101250.0
1,F,N,Y,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,0,1,...,41,10.0,3.0,5,9.0,2.0,35,1.0,0.0,101250.0
2,F,Y,Y,157500.0,State servant,Higher education,Married,House / apartment,0,0,...,41,9.0,0.0,5,10.0,0.0,35,11.0,3.0,78750.0
3,F,N,Y,270000.0,Working,Secondary / secondary special,Married,House / apartment,0,0,...,36,3.0,0.0,13,10.0,1.0,23,4.0,2.0,67500.0
4,F,N,N,315000.0,Working,Secondary / secondary special,Separated,House / apartment,0,0,...,48,9.0,2.0,5,5.0,2.0,42,3.0,3.0,315000.0


In [41]:
test.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
index,,,,,,,,,,,,,,,,,,,,,
26457,M,Y,N,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,0,1,...,60,1.0,1.0,0,0.0,0.0,60,1.0,1.0,56250.0
26458,F,N,Y,135000.0,State servant,Higher education,Married,House / apartment,0,1,...,51,8.0,1.0,23,1.0,2.0,28,7.0,2.0,67500.0
26459,F,N,Y,69372.0,Working,Secondary / secondary special,Married,House / apartment,1,1,...,43,1.0,1.0,0,7.0,3.0,42,6.0,2.0,34686.0
26460,M,Y,N,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,...,52,6.0,0.0,6,0.0,1.0,45,5.0,3.0,56250.0
26461,F,Y,Y,225000.0,State servant,Higher education,Married,House / apartment,1,0,...,48,6.0,2.0,25,0.0,0.0,23,5.0,1.0,112500.0


### Feature Engineering

In [7]:
for x in [train, test]:
    # Just adding some features about unemployment:
    x['days_unemployed'] = x['days_birth'] - x['days_employed']
    x['income_unemployed'] = x['income_total'] / x['days_unemployed']

    # Adding Age, birth month and birth week. Birth_month: Month when the person born, etc.
    x['Age'] = x['days_birth'] // 365
    x['birth_month'] = np.floor(x['days_birth'] / 30) - ((np.floor(x['days_birth'] / 30) / 12).astype(int) * 12)
    x['birth_week'] = np.floor(x['days_birth'] / 7) - ((np.floor(x['days_birth'] / 7) / 4).astype(int) * 4)

    # Same with days_birth but with days_employed.
    x['ages_employed'] = x['days_employed'] // 365
    x['employ_month'] = np.floor(x['days_employed'] / 30) - ((np.floor(x['days_employed'] / 30) / 12).astype(int) * 12)
    x['employ_week'] = np.floor(x['days_employed'] / 7) - ((np.floor(x['days_employed'] / 7) / 4).astype(int) * 4)

    # Same but with days unemployed:
    x['ages_unemployed'] = x['days_unemployed'] // 365
    x['unemploy_month'] = np.floor(x['days_unemployed'] / 30) - ((np.floor(x['days_unemployed'] / 30) / 12).astype(int) * 12)
    x['unemploy_week'] = np.floor(x['days_unemployed'] / 7) - ((np.floor(x['days_unemployed'] / 7) / 4).astype(int) * 4)

    # Actual income per family member.
    x["income_family"] = x["income_total"] / x["family_size"]

    # Handling with multicollinearity
    cols = ['child_num', 'days_birth', 'days_employed']
    x.drop(cols, axis=1, inplace=True)

In [42]:
train.columns

Index(['gender', 'car', 'reality', 'income_total', 'income_type', 'edu_type',
       'family_type', 'house_type', 'work_phone', 'home_phone', 'email',
       'occup_type', 'family_size', 'begin_month', 'credit', 'days_unemployed',
       'income_unemployed', 'Age', 'birth_month', 'birth_week',
       'ages_employed', 'employ_month', 'employ_week', 'ages_unemployed',
       'unemploy_month', 'unemploy_week', 'income_family'],
      dtype='object')

In [43]:
test.columns

Index(['gender', 'car', 'reality', 'income_total', 'income_type', 'edu_type',
       'family_type', 'house_type', 'work_phone', 'home_phone', 'email',
       'occup_type', 'family_size', 'begin_month', 'days_unemployed',
       'income_unemployed', 'Age', 'birth_month', 'birth_week',
       'ages_employed', 'employ_month', 'employ_week', 'ages_unemployed',
       'unemploy_month', 'unemploy_week', 'income_family'],
      dtype='object')

### Encoding

In [44]:
# Separate numerical and categorical features
num = train.dtypes[train.dtypes != "object"].index.tolist()
num.remove('credit')

cat = train.dtypes[train.dtypes == "object"].index.tolist()

In [10]:
num

['income_total',
 'work_phone',
 'home_phone',
 'email',
 'family_size',
 'begin_month',
 'days_unemployed',
 'income_unemployed',
 'Age',
 'birth_month',
 'birth_week',
 'ages_employed',
 'employ_month',
 'employ_week',
 'ages_unemployed',
 'unemploy_month',
 'unemploy_week',
 'income_family']

In [11]:
cat

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occup_type']

In [45]:
# StandardScaler
scaler = StandardScaler()
train[num] = scaler.fit_transform(train[num])
test[num] = scaler.transform(test[num])

The result using OrdinalEncoding and LabelEncoding is exactly the same, there's no difference:

In [46]:
# OrdinalEncoding
oenc = OrdinalEncoder()
train[cat] = oenc.fit_transform(train[cat], train['credit'])
test[cat] = oenc.transform(test[cat])

In [139]:
# THIS CODE IS DEPRECATED AND SHOULD NOT BE USED ANYMORE

# LabelEncoding
for x in cat:
  lenc = LabelEncoder()
  lenc = lenc.fit(train[x])
  train[x] = lenc.transform(train[x])
  test[x] = lenc.transform(test[x])

## **Model and train**

In [47]:
def stratified_kfold_rf(p, n_fold, X, y, X_test):
    # Declaring Stratified K-Fold:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    # Initializing arrays:
    rf_oof = np.zeros((X.shape[0], 3))
    rf_preds = np.zeros((X_test.shape[0], 3))

    # Main loop:
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Model fit:
        model = RandomForestClassifier(**p)
        model.fit(X_train, y_train)

        rf_oof[valid_idx] = model.predict_proba(X_valid) # Infers the class probability of train dataset
        rf_preds += model.predict_proba(X_test) / n_fold # Infers the class probability of test dataset
        print(f"Log Loss Score: {log_loss(y_valid, rf_oof[valid_idx]):.5f}")

    log_score = log_loss(y, rf_oof)
    print(f"\nLog Loss Score: {log_score:.5f}")

    return rf_oof, rf_preds

In [48]:
# Manual hyperparameterization
p = {
  "criterion": "entropy",
  "n_estimators": 300,
  "min_samples_split": 10,
  "min_samples_leaf": 2,
  "max_features": "sqrt",
  "oob_score": True,
  "random_state": 42,
  "n_jobs": -1,
  }

In [49]:
X = train.drop("credit", axis=1)
y = train["credit"]
X_test = test.copy()

In [50]:
rf_oof, rf_preds = stratified_kfold_rf(p, 10, X, y, X_test)

============ Fold 0 ============
Log Loss Score: 0.68736
============ Fold 1 ============
Log Loss Score: 0.68164
============ Fold 2 ============
Log Loss Score: 0.66641
============ Fold 3 ============
Log Loss Score: 0.66620
============ Fold 4 ============
Log Loss Score: 0.68641
============ Fold 5 ============
Log Loss Score: 0.67191
============ Fold 6 ============
Log Loss Score: 0.70074
============ Fold 7 ============
Log Loss Score: 0.68104
============ Fold 8 ============
Log Loss Score: 0.68441
============ Fold 9 ============
Log Loss Score: 0.69283

Log Loss Score: 0.68189


Now, let's try using Optuna to find best hyperparameters:

In [51]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    p = {
        "max_depth": trial.suggest_int('max_depth', 1, 10),
        "max_leaf_nodes": trial.suggest_int('max_leaf_nodes', 2, 1000),
        "n_estimators": trial.suggest_int('n_estimators', 100, 500),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 30),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 10),
    }

    rf = RandomForestClassifier(**p)
    rf.fit(X, y)

    score = cross_val_score(rf, X, y, cv=5, scoring="neg_log_loss")
    log_loss = score.mean()

    return log_loss

In [52]:
import optuna

rf_study = optuna.create_study(direction='maximize')
rf_study.optimize(objective, n_trials=10)

print("Best Score:",rf_study.best_value)
print("Best trial",rf_study.best_trial.params)

[I 2022-12-14 22:02:01,585] A new study created in memory with name: no-name-bf17c096-9558-45b6-91aa-2ab4de3ca123
[I 2022-12-14 22:02:10,066] Trial 0 finished with value: -0.8472836619682053 and parameters: {'max_depth': 1, 'max_leaf_nodes': 156, 'n_estimators': 423, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: -0.8472836619682053.
[I 2022-12-14 22:02:36,148] Trial 1 finished with value: -0.7832537702382482 and parameters: {'max_depth': 8, 'max_leaf_nodes': 809, 'n_estimators': 332, 'min_samples_split': 14, 'min_samples_leaf': 2}. Best is trial 1 with value: -0.7832537702382482.
[I 2022-12-14 22:03:03,304] Trial 2 finished with value: -0.7896294025031103 and parameters: {'max_depth': 7, 'max_leaf_nodes': 284, 'n_estimators': 376, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 1 with value: -0.7832537702382482.
[I 2022-12-14 22:03:21,764] Trial 3 finished with value: -0.8205707610991295 and parameters: {'max_depth': 3, 'max_leaf_nodes': 902,

Best Score: -0.7700081934941353
Best trial {'max_depth': 10, 'max_leaf_nodes': 431, 'n_estimators': 466, 'min_samples_split': 25, 'min_samples_leaf': 1}


In [55]:
# Optuna hyperparameterization
p_optuna = {
  "criterion": "entropy",
  "max_features": "sqrt",
  "oob_score": True,
  "random_state": 42,
  "n_jobs": -1,

  # "max_leaf_nodes": 431,
  "n_estimators": 466,
  "min_samples_split": 25,
  "min_samples_leaf": 1,
  }

In [56]:
rf_oof_optuna, rf_preds_optuna = stratified_kfold_rf(p_optuna, 10, X, y, X_test)

============ Fold 0 ============
Log Loss Score: 0.70227
============ Fold 1 ============
Log Loss Score: 0.69876
============ Fold 2 ============
Log Loss Score: 0.68893
============ Fold 3 ============
Log Loss Score: 0.68872
============ Fold 4 ============
Log Loss Score: 0.70318
============ Fold 5 ============
Log Loss Score: 0.69329
============ Fold 6 ============
Log Loss Score: 0.71338
============ Fold 7 ============
Log Loss Score: 0.69996
============ Fold 8 ============
Log Loss Score: 0.69830
============ Fold 9 ============
Log Loss Score: 0.70844

Log Loss Score: 0.69952


The result using manual hyperparameterization is better, so let's save this configuration:

In [19]:
rf_preds.shape

(10000, 3)

In [20]:
rf_oof.shape

(23392, 3)

## **Save results**

In [21]:
submission = pd.read_csv("../data/sample_submission.csv")
submission.set_index("index", inplace=True)
submission[:] = rf_preds
submission.head()

,0,1,2
index,,,
26457,0.068513,0.216909,0.714578
26458,0.303931,0.274497,0.421573
26459,0.039117,0.066872,0.894011
26460,0.081644,0.083682,0.834674
26461,0.114849,0.172174,0.712977
